In [23]:
%reset -f

In [24]:
#!/usr/bin/env python3
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import torch
from torch import nn
from torch.utils.data import DataLoader
from torcheval.metrics import BinaryAccuracy
from torcheval.metrics.functional import binary_accuracy
torch.manual_seed(18)
torch.cuda.is_available()
from FaceNetV3 import FaceNetV3, ConvBlock
from FacesDataset import FacesDataset

In [25]:
def epoch_system_out_string(epoch:int, train_loss:float, train_acc:float, val_loss:float, val_acc:float, test_acc:float)->str:
    return (f'Epoch: {epoch} -- train Loss: {round(train_loss, 4)} \t valid Loss: {round(val_loss, 4)} \t train acc.:{round(train_acc, 4)} \t val acc.:{round(val_acc, 4)} \t test acc.:{round(test_acc, 4)}')

In [26]:
@torch.no_grad()
def estimate_performance(model, data_loader, device='cuda'):
    loss_value = 0.0
    acc_value = 0.0
    loss_fn = nn.BCELoss()
    loss_fn.to(device)
    metric = BinaryAccuracy()
    metric.to(device)
    model.eval()
    for batch in tqdm(data_loader):
        inputs, targets = batch
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs.flatten().float(), targets.float())
        metric.update(outputs.flatten(), targets)
        loss_value += loss.item()
        acc_value += metric.compute().item()
    return loss_value/data_loader.__len__(), acc_value/data_loader.__len__()

In [27]:
def train(model, optimizer, train_loader, val_loader, test_loader, epochs, loss_fn, device='cuda'):
    model.to(device)
    loss_fn.to(device)
    metric = BinaryAccuracy()
    metric.to(device)
    for epoch in (range(epochs)):
        # train the model on the training set
        train_loss = 0.0
        train_acc = 0.0
        model.train()
        for batch in tqdm(train_loader):
            optimizer.zero_grad()
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = loss_fn(outputs.flatten().float(), targets.float())
            loss.backward()
            optimizer.step()
            metric.update(outputs.flatten(), targets)
            train_loss += loss.item()
            train_acc += metric.compute().item()

        # estimate the performance of the model on the validation set and the test set
        train_loss, train_acc = train_loss / len(train_loader), train_acc / len(train_loader)
        val_loss, val_acc = estimate_performance(model, val_loader, device)
        test_loss, test_acc = estimate_performance(model, test_loader, device)

        print(epoch_system_out_string(epoch, train_loss, train_acc, val_loss, val_acc, test_acc))

In [28]:
batch_size = 32

In [29]:
training_set = FacesDataset('datasets/train.csv')
train_loader = DataLoader(dataset=training_set, batch_size=batch_size, shuffle=True)

valid_set = FacesDataset('datasets/valid.csv')
valid_loader = DataLoader(dataset=valid_set, batch_size=batch_size, shuffle=True)

test_set = FacesDataset('datasets/test.csv')
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

In [30]:
model = FaceNetV3()

In [31]:
model.forward(torch.randn(1, 3, 256, 256)), model.forward(torch.randn(1, 3, 256, 256)).size()

(tensor([[0.4593]], grad_fn=<SigmoidBackward0>), torch.Size([1, 1]))

In [32]:
loss_fn = nn.BCELoss()

params = model.parameters()
learning_rate = 3e-4
optimizer = torch.optim.Adam(params, lr=learning_rate)

# train the model
num_epochs = 15
train(model=model, optimizer=optimizer, train_loader=train_loader, val_loader=valid_loader, test_loader=test_loader,loss_fn=loss_fn, epochs=num_epochs)

100%|██████████| 313/313 [00:21<00:00, 14.36it/s]


Epoch: 0 -- train Loss: 0.6057 	 valid Loss: 0.5282 	 train acc.:0.5758 	 val acc.:0.7432 	 test acc.:0.7418


100%|██████████| 313/313 [00:21<00:00, 14.36it/s]


Epoch: 1 -- train Loss: 0.414 	 valid Loss: 0.3573 	 train acc.:0.6976 	 val acc.:0.8462 	 test acc.:0.8491


100%|██████████| 313/313 [00:21<00:00, 14.34it/s]


...

Epoch: 12 -- train Loss: 0.1145 	 valid Loss: 0.1829 	 train acc.:0.8936 	 val acc.:0.9334 	 test acc.:0.9302


100%|██████████| 313/313 [00:21<00:00, 14.31it/s]


Epoch: 13 -- train Loss: 0.1055 	 valid Loss: 0.1514 	 train acc.:0.8983 	 val acc.:0.945 	 test acc.:0.9424


100%|██████████| 313/313 [00:21<00:00, 14.24it/s]

Epoch: 14 -- train Loss: 0.1015 	 valid Loss: 0.1683 	 train acc.:0.9026 	 val acc.:0.938 	 test acc.:0.9425


In [33]:
model.save()

Saving model to models/FaceNetV3-2023-03-28-22-53-51.pth
Done!
